# **Gomoku : battle IA Vs Humain**
Pierre NIETO
Jolann NARCISSE
Clement
Maxime MOTTIER

# Création du plateau


1.   Crée une grille vide où chaque case contient un point (.).
2.   Remplit les cases avec X pour les mouvements du joueur humain et O pour ceux de l'IA, en fonction de l'état du jeu (un dictionnaire etat qui stocke les positions jouées).
3.   Affiche les indices de colonnes en chiffres (0 à 14) et les indices de lignes en lettres (A à O).





In [1]:
def afficher_planche(etat):
    grille = [['.' for _ in range(15)] for _ in range(15)]

    for (x, y), valeur in etat.items():
        grille[x][y] = 'X' if valeur == 1 else 'O'

    print('    ' + '  '.join([f'{i:2}' for i in range(15)]))

    for i in range(15):
        print(f'{chr(i + 65):2}  ' + '   '.join(grille[i]))

# Vérification de la victoire


1.   Parcourt les positions jouées par un joueur donné.

2.   Vérifie les alignements dans 4 directions possibles :
  *   Horizontal : Cases adjacentes sur une ligne.
  *   Vertical : Cases adjacentes sur une colonne.
  *   Diagonale : Alignement en diagonale descendante.
  * Anti-diagonale : Alignement en diagonale montante.


3. Retourne True si une des conditions est satisfaite.





In [2]:
def gagner(etat, joueur):
    for x, y in [i for i in etat.keys() if etat[i] == joueur]:
        if all((x + i, y) in etat and etat[(x + i, y)] == joueur for i in range(5)):
            return True
        if all((x, y + i) in etat and etat[(x, y + i)] == joueur for i in range(5)):
            return True
        if all((x + i, y + i) in etat and etat[(x + i, y + i)] == joueur for i in range(5)):
            return True
        if all((x - i, y + i) in etat and etat[(x - i, y + i)] == joueur for i in range(5)):
            return True
    return False

# Conversion d’un coup en coordonnées

1. Sépare la lettre (colonne) et le chiffre (ligne) du coup.
2. Utilise un dictionnaire pour convertir la lettre en un numéro (A=0, B=1, ...).
3. Retourne un tuple (colonne, ligne).

In [3]:
def coupintocoord(move):
    dicto_lettres_chiffres = {chr(65 + i): i for i in range(15)}
    try:
        Lint = ''.join(filter(str.isalpha, move))
        L = dicto_lettres_chiffres[Lint]
        C = int(''.join(filter(str.isdigit, move)))
        return L, C
    except (KeyError, ValueError):
        raise ValueError("Le coup doit être une chaîne de caractères valide, par exemple 'H7'.")

# Jouer un coup
1. Convertit le coup en coordonnées avec coupintocoord.
2. Si la position est libre, elle est ajoutée à l'état du jeu :
  * 1 pour le joueur humain.
  * 0 pour l’IA.

In [4]:
def jouer(etat, move, IA):
    coord = coupintocoord(move)
    position = (coord[0], coord[1])
    if position not in etat.keys():
        etat[position] = 0 if IA else 1

# Vérification des règles des premiers tours
1. **premiertour** : Vérifie si le premier coup est "H7" (case centrale obligatoire).
2. **secondtour** : Vérifie si le deuxième coup est suffisamment éloigné de "H7" (au moins 3 cases dans les deux directions).


In [5]:
def premiertour(coup):
    return coup == "H7"

def secondtour(coup):
    coord = coupintocoord(coup)
    l = coord[0]
    c = coord[1]
    if abs(l - 7) > 3 and abs(c - 7) > 3:
        return True
    return False

# Saisie et validation des coups

1. **demandercoup** : Demande à l'utilisateur d'entrer un coup.
2. **verifiercoup** : Vérifie si le coup est valide :
  * Les coordonnées doivent être dans les limites du plateau.
  * La case ne doit pas déjà être occupée.

In [6]:
def demandercoup():
    coup = input("Entrer la case que vous voulez jouer: ").capitalize()
    return coup

def verifiercoup(etat, coup):
    coord = coupintocoord(coup)
    l = coord[0]
    c = coord[1]
    position = (l, c)
    if (l < 0 or l > 14) or (c < 0 or c > 14):
        return False
    elif position in etat.keys():
        return False
    return True

def couptotal(etat):
    coup = demandercoup()
    while not verifiercoup(etat, coup):
        coup = demandercoup()
    return coup

# Recherche de coups par l’IA

On a utilise minmax pour déterminer le meilleur coup, puis l'algo alphabeta pour supprimer les branches de possibilités inutiles

1. **minmax** :
Explore tous les coups possibles jusqu’à une certaine profondeur.
Maximiser le score pour l’IA tout en minimisant celui du joueur adverse.
2. **alphabeta** :
Une version optimisée de minmax.
Utilise deux bornes (alpha et beta) pour réduire les calculs.

In [7]:
def minmax(etat, profondeur, maximiser, joueur):
    if profondeur == 0 or gagner(etat, joueur) or gagner(etat, 1 - joueur):
        return evaluation(etat, joueur), None

    meilleur_coup = None
    meilleurscore = -float('inf') if maximiser else float('inf')

    for voisin in voisins_libres(etat):
        etat[voisin] = joueur if maximiser else 1 - joueur
        score, _ = minmax(etat, profondeur - 1, not maximiser, joueur)
        del etat[voisin]

        if maximiser:
            if score > meilleurscore:
                meilleurscore = score
                meilleur_coup = voisin
        else:
            if score < meilleurscore:
                meilleurscore = score
                meilleur_coup = voisin

    return meilleurscore, meilleur_coup

def alphabeta(etat, profondeur, alpha, beta, maximiser, joueur):
    if profondeur == 0 or gagner(etat, joueur) or gagner(etat, 1 - joueur):
        return evaluation(etat, joueur), None

    meilleur_coup = None

    if maximiser:
        meilleurscore = -float('inf')
        for voisin in voisins_libres(etat):
            etat[voisin] = joueur
            score, _ = alphabeta(etat, profondeur - 1, alpha, beta, False, joueur)
            del etat[voisin]

            if score > meilleurscore:
                meilleurscore = score
                meilleur_coup = voisin

            alpha = max(alpha, score)
            if beta <= alpha:
                break
        return meilleurscore, meilleur_coup
    else:
        meilleurscore = float('inf')
        for voisin in voisins_libres(etat):
            etat[voisin] = 1 - joueur
            score, _ = alphabeta(etat, profondeur - 1, alpha, beta, True, joueur)
            del etat[voisin]

            if score < meilleurscore:
                meilleurscore = score
                meilleur_coup = voisin

            beta = min(beta, score)
            if beta <= alpha:
                break
        return meilleurscore, meilleur_coup

# Évaluation de l’état du jeu

1. Parcourt toutes les directions depuis chaque position jouée.
2. Calcule des points pour :
  * Les alignements favorables au joueur donné.
  * Les alignements de l’adversaire (pour anticiper les menaces).
3. Plus un alignement est long, plus il rapporte de points (cubés : alignement^3).

In [8]:
def evaluation(etat, joueur):
    score = 0
    directions = [(1, 0), (0, 1), (1, 1), (1, -1)]
    adversaire = 1 - joueur

    for (x, y), valeur in etat.items():
        for dx, dy in directions:
            alignement_joueur = 0
            alignement_adversaire = 0

            for i in range(5):
                nx, ny = x + dx * i, y + dy * i
                if (nx, ny) in etat and etat[(nx, ny)] == joueur:
                    alignement_joueur += 1
                elif (nx, ny) in etat and etat[(nx, ny)] == adversaire:
                    alignement_joueur = 0
                    break

            for i in range(5):
                nx, ny = x + dx * i, y + dy * i
                if (nx, ny) in etat and etat[(nx, ny)] == adversaire:
                    alignement_adversaire += 1
                elif (nx, ny) in etat and etat[(nx, ny)] == joueur:
                    alignement_adversaire = 0
                    break

            score += alignement_joueur ** 3
            score -= alignement_adversaire ** 3

    return score

# Recherche des voisins libres

1. Pour chaque case occupée, regarde les 8 cases voisines (haut, bas, gauche, droite, diagonales).
2. Filtre les cases hors limites ou déjà occupées.

In [9]:
def voisins_libres(etat):
    voisins = set()
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (1, 1), (-1, 1), (1, -1)]

    for (x, y) in etat.keys():
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if (nx, ny) not in etat and 0 <= nx < 15 and 0 <= ny < 15:
                voisins.add((nx, ny))
    return voisins

# Le MAIN
1. Initialise un état vide (etat).
2. Affiche la planche initiale.
3. Gère les tours alternés entre le joueur humain et l’IA.
  * Ajout de la possibilité de choisir qui commence le jeu entre le joueur et l'IA.
  * Ajout de la gestion des coups spécifiques pour le premier et le deuxième tour pour les deux parties (joueur et IA).
  * La gestion du premier et du deuxième tour est désormais explicitement demandée à l'IA ou au joueur.
  * vérification si les deux premiers tous verifient les règles.
  * L’IA choisit son coup à l’aide de l’algorithme choisi (alphabeta ou minmax).
  
4. Affiche la planche après chaque coup.
5. Vérifie après chaque tour si l’un des joueurs a gagné.

In [ ]:
def main():
    etat = {}
    afficher_planche(etat)
    tour = 0
    ia = 0
    joueur = 1
    algo = "alphabeta"

    premier_joueur = input("\nQui commence ? Tapez 'joueur' ou 'ia': ").strip().lower()
    while premier_joueur not in ['joueur', 'ia']:
        premier_joueur = input("Entrée invalide. Tapez 'joueur' ou 'ia': ").strip().lower()

    ia_commence = premier_joueur == 'ia'

    if ia_commence:
        coup_ia = (7, 7)
        etat[coup_ia] = ia
        afficher_planche(etat)
        tour += 1

    while not gagner(etat, joueur) and not gagner(etat, ia):
        if not ia_commence or tour > 0:
            coup = couptotal(etat)

            if not ia_commence and tour == 0:
                if not premiertour(coup):
                    print("Votre coup doit être 'H7' pour le premier tour.")
                    continue

            if (ia_commence and tour == 2) or (not ia_commence and tour == 1):
                if not secondtour(coup):
                    print("Votre coup doit être à au moins 3 cases de 'H7' pour le deuxième tour.")
                    continue

            jouer(etat, coup, False)
            afficher_planche(etat)

        if gagner(etat, joueur):
            print("Victoire du joueur !")
            break

        coup_ia = None
        if algo == "minmax":
            _, coup_ia = minmax(etat, 3, True, ia)
        else:
            _, coup_ia = alphabeta(etat, 3, -float('inf'), float('inf'), True, ia)

        if ia_commence and tour == 0:
            coup_ia = (7, 7)
        elif ia_commence and tour == 1:
            valid_moves = [voisin for voisin in voisins_libres(etat)
                           if abs(voisin[0] - 7) > 3 and abs(voisin[1] - 7) > 3]
            if valid_moves:
                coup_ia = valid_moves[0]

        if coup_ia is None or coup_ia in etat:
            for case_libre in voisins_libres(etat):
                if case_libre not in etat:
                    coup_ia = case_libre
                    break

        if coup_ia:
            etat[coup_ia] = ia
            afficher_planche(etat)

        if gagner(etat, ia):
            print("Victoire de l'ia!")
            break

        tour += 1

main()

     0   1   2   3   4   5   6   7   8   9  10  11  12  13  14
A   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .
B   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .
C   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .
D   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .
E   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .
F   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .
G   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .
H   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .
I   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .
J   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .
K   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .
L   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .
M   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .
N   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .
O   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .

Qui co